In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, uniform
from kan import *
import torch
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
plt.rcParams.update({
    'font.size': 12,               # Font size
    'axes.labelsize': 15,          # Label size
    'axes.titlesize': 15,          # Title size
    'axes.linewidth': 1.2,         # Axis line width
    'xtick.labelsize': 10,         # Size of x-axis tick labels
    'ytick.labelsize': 10,         # Size of y-axis tick labels
    'xtick.major.size': 6,         # Length of major ticks on the x-axis
    'ytick.major.size': 6,         # Length of major ticks on the y-axis
    'xtick.major.width': 1.0,      # Width of major ticks on the x-axis
    'ytick.major.width': 1.0,      # Width of major ticks on the y-axis
    'figure.dpi': 300,             # Image resolution
    'savefig.dpi': 300,            # Resolution for saving figures
    'figure.figsize': (12, 5),     # Figure size
})

## Scenario A1

### $I_{u, x} \mid \, Y_{x} > u \sim \text{Bern}\{m_{\text{A}1}(x; u)\}$

In [ ]:
def MC_A1(M, n):
    x_cl = np.linspace(0, 1, 1000)
    u_quantile = 0.95
    probabilities_all = []

    for i in range(M):
        np.random.seed(i)

        # X ~ U(0,1)
        X = uniform.rvs(loc=0, scale=1, size=n)

        # Y_X ~ Unit Fréchet
        U = np.random.uniform(0, 1, size=n)
        Y_X = 1 / -np.log(n / (n + 1) * U)

        u  = np.quantile(Y_X, u_quantile)
        mask = Y_X > u
        X_masked = X[mask]
        Phi_X_masked = norm.cdf(X_masked, loc=np.exp(-u), scale=1)
        I_X = np.zeros(n, dtype=int)
        I_X[mask] = np.random.binomial(1, Phi_X_masked)

        # Dataframe
        df = pd.DataFrame({'X': X, 'Y_X': Y_X, 'I_X': I_X})
        df_sorted = df.sort_values('X')
        df_sim = df_sorted[df_sorted['Y_X'] > u]

        # Neural inputs
        x_train_cl = df_sim['X'].values
        y_train_cl = df_sim['I_X'].values

        x_train_tensor = torch.tensor(x_train_cl, dtype=torch.float32).view(-1, 1)
        y_train_tensor = torch.tensor(y_train_cl, dtype=torch.long)

        model = KAN(width=[1, 2], grid=2, k=2)

        dataset = {
            'train_input': x_train_tensor,
            'train_label': y_train_tensor,
            'test_input': x_train_tensor,
            'test_label': y_train_tensor
        }

        model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())

        x_cl_tensor = torch.tensor(x_cl, dtype=torch.float32).view(-1, 1) 
        with torch.no_grad():
            Z = model(x_cl_tensor)
            Z = torch.softmax(Z, dim=1)
            probabilities1 = Z[:, 1].numpy()
        
        probabilities_all.append(probabilities1)

    probabilities_all = np.array(probabilities_all)
    
    return probabilities_all, x_cl


In [ ]:
probabilities_all_1, x_cl = MC_A1(500, 5000)

In [ ]:
monte_carlo_mean = np.mean(probabilities_all_1, axis=0)
Phi_x_cl = norm.cdf(x_cl)

squared_difference = (monte_carlo_mean - Phi_x_cl) ** 2
mise = np.trapz(squared_difference, x_cl)

print('MISE: ' + str(mise))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

for probs in probabilities_all_1:
    ax.plot(x_cl, probs, color='gray', linewidth=1, alpha = 0.1)

ax.plot(x_cl, monte_carlo_mean, color='red', linewidth=2)

Phi_x_cl = norm.cdf(x_cl)
ax.plot(x_cl, Phi_x_cl, color='black', linewidth=2, linestyle='--')

ax.set_xlabel(r'$x$', fontsize=12) 
ax.set_ylabel(r'$\alpha_I(x)$', fontsize=12)
ax.set_ylim(0, 1)

plt.show()

## Scenario A2

### $I_{u, x} \mid \, Y_{x} > u \sim \text{Bern}\{m_{\text{A}2}(x; u)\}$

In [ ]:
def MC_A2(M, n):
    x_cl = np.linspace(0, 1, 1000)
    u_quantile = 0.95
    probabilities_all = []

    for i in range(M):
        np.random.seed(i)

        # X ~ U(0,1)
        X = uniform.rvs(loc=0, scale=1, size=n)

        # Y_X ~ Unit Fréchet
        U = np.random.uniform(0, 1, size=n)
        Y_X = 1 / -np.log(n / (n + 1) * U)

        u  = np.quantile(Y_X, u_quantile)

        def f(x):
            return 0.2 * np.sin(3 * np.pi * (x - 1)**1) + 0.4 + 1/(u**2)

        mask = Y_X > u
        X_masked = X[mask]
        f_X_masked = f(X_masked)
        I_X = np.zeros(n, dtype=int)
        I_X[mask] = np.random.binomial(1, f_X_masked)

        # Dataframe
        df = pd.DataFrame({'X': X, 'Y_X': Y_X, 'I_X': I_X})
        df_sorted = df.sort_values('X')
        df_sim = df_sorted[df_sorted['Y_X'] > u]

        # Neural inputs
        x_train_cl = df_sim['X'].values
        y_train_cl = df_sim['I_X'].values

        x_train_tensor = torch.tensor(x_train_cl, dtype=torch.float32).view(-1, 1)
        y_train_tensor = torch.tensor(y_train_cl, dtype=torch.long)

        model = KAN(width=[1, 2], grid=2, k=3)

        dataset = {
            'train_input': x_train_tensor,
            'train_label': y_train_tensor,
            'test_input': x_train_tensor,
            'test_label': y_train_tensor
        }

        model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())

        x_cl_tensor = torch.tensor(x_cl, dtype=torch.float32).view(-1, 1)
        with torch.no_grad():
            Z = model(x_cl_tensor)
            Z = torch.softmax(Z, dim=1)
            probabilities1 = Z[:, 1].numpy()
        
        probabilities_all.append(probabilities1)

    probabilities_all = np.array(probabilities_all)
    
    return probabilities_all, x_cl

In [ ]:
probabilities_all_2, x_cl = MC_A2(500, 15000)

In [ ]:
def f(x):
    return 0.2 * np.sin(3 * np.pi * (x - 1)**1) + 0.4

monte_carlo_mean_2 = np.mean(probabilities_all_2, axis=0)
f_x_cl = f(x_cl)

squared_difference_2 = (monte_carlo_mean_2 - f_x_cl) ** 2
mise_2 = np.trapz(squared_difference_2, x_cl)

print('MISE: ' + str(mise_2))

fig, ax = plt.subplots(figsize=(6, 4))

for probs in probabilities_all_2:
    ax.plot(x_cl, probs, color='gray', linewidth=1, alpha = 0.1) 

ax.plot(x_cl, monte_carlo_mean_2, color='red', linewidth=2)

ax.plot(x_cl, f_x_cl, color='black', linewidth=2, linestyle='--')

ax.set_xlabel(r'$x$', fontsize=12) 
ax.set_ylabel(r'$\alpha_I(x)$', fontsize=12)
ax.set_ylim(0, 1)

plt.show()